# Notebook for testing purposes

In [26]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import datetime
import plotly.graph_objects as ly
import os
from scipy.signal import savgol_filter, find_peaks
import matplotlib.pyplot as plt
from classes import Market

In [27]:
os.getcwd()

'c:\\Users\\adame\\OneDrive\\Bureau\\CODE\\CandleMagic'

In [28]:
m = Market("data\\EURUSD_M1_231201_231215.csv")

In [29]:
m.data[m.data["ts"] == datetime.datetime(2023, 12, 13, 2,19, 0)]

ts     open    high      low   close  <TICKVOL>  \
12077 2023-12-13 02:19:00  1.07814  1.0782  1.07814  1.0782         21   

            SMA       EMA Uptrend Downtrend  
12077  1.078255  1.078241   False     False

In [30]:
import bisect
L = [1,3,5,6,8,9]
L.insert(bisect.bisect(L, 7), 7)
L

[1, 3, 5, 6, 7, 8, 9]

In [31]:
m.data["ts"].map(lambda x: str(x)[:10]).unique()

array(['2023-11-30', '2023-12-01', '2023-12-03', '2023-12-04',
       '2023-12-05', '2023-12-06', '2023-12-07', '2023-12-08',
       '2023-12-10', '2023-12-11', '2023-12-12', '2023-12-13',
       '2023-12-14', '2023-12-15'], dtype=object)

In [32]:
m.data['ts2'] = m.data['ts'].map(lambda x: x - datetime.timedelta(hours = 7))

In [33]:
m.data

ts     open     high      low    close  <TICKVOL>  \
0     2023-11-30 17:00:00  1.08845  1.08845  1.08845  1.08845          1   
1     2023-11-30 17:01:00  1.08844  1.08877  1.08768  1.08808         11   
2     2023-11-30 17:02:00  1.08808  1.08821  1.08764  1.08820         24   
3     2023-11-30 17:03:00  1.08830  1.08830  1.08820  1.08820          2   
4     2023-11-30 17:04:00  1.08830  1.08835  1.08830  1.08835          2   
...                   ...      ...      ...      ...      ...        ...   
15517 2023-12-15 11:39:00  1.09091  1.09092  1.09074  1.09090        231   
15518 2023-12-15 11:40:00  1.09090  1.09098  1.09079  1.09084        250   
15519 2023-12-15 11:41:00  1.09084  1.09091  1.09062  1.09072        251   
15520 2023-12-15 11:42:00  1.09073  1.09075  1.09049  1.09068        262   
15521 2023-12-15 11:43:00  1.09070  1.09072  1.09055  1.09056        228   

            SMA       EMA Uptrend Downtrend                 ts2  
0           NaN  1.088450     NaN       NaN 2023-11-30 10:00:00  
1           NaN  1.088508     NaN       NaN 2023-11-30 10:01:00  
2           NaN  1.088454     NaN       NaN 2023-11-30 10:02:00  
3           NaN  1.088426     NaN       NaN 2023-11-30 10:03:00  
4           NaN  1.088412     NaN       NaN 2023-11-30 10:04:00  
...         ...       ...     ...       ...                 ...  
15517  1.091462  1.091359   False     False 2023-12-15 04:39:00  
15518  1.091411  1.091290   False     False 2023-12-15 04:40:00  
15519  1.091344  1.091221   False     False 2023-12-15 04:41:00  
15520  1.091251  1.091136    True     False 2023-12-15 04:42:00  
15521  1.091164  1.091060    True     False 2023-12-15 04:43:00  

[15522 rows x 11 columns]

In [34]:
days = m.data["ts"].map(lambda x: str(x.month) + "-" + str(x.day))

In [35]:
days.unique()

array(['11-30', '12-1', '12-3', '12-4', '12-5', '12-6', '12-7', '12-8',
       '12-10', '12-11', '12-12', '12-13', '12-14', '12-15'], dtype=object)

In [36]:
file = "data\\EURUSD_M1_231201_231215.csv"
df = pd.read_csv(file, sep="\t")
df['datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'], format='%Y.%m.%d %H:%M:%S')

df = df.rename({"<CLOSE>": "close",
           "<OPEN>": "open",
           "<HIGH>": "high",
           "<LOW>": "low"
           }, axis = 1)

df = df.drop(['<DATE>', '<TIME>', '<VOL>', '<SPREAD>' ], axis=1)
df = df[['datetime'] + [col for col in df.columns if col != 'datetime']]
df["atr"] = ta.atr(high = df["high"], low = df["low"], close = df["close"])
df["atr"] = df["atr"].rolling(window=30).mean()
n_candles = df.shape[0]

In [37]:
df['SMA'] = df['close'].rolling(window=10).mean()
df['EMA'] = df['close'].ewm(span=10, adjust=False).mean()

In [38]:
df.iloc[50:60]

datetime     open     high      low    close  <TICKVOL>  \
50 2023-12-01 00:50:00  1.08863  1.08864  1.08862  1.08864          5   
51 2023-12-01 00:51:00  1.08863  1.08864  1.08862  1.08863          6   
52 2023-12-01 00:52:00  1.08852  1.08865  1.08852  1.08863          5   
53 2023-12-01 00:53:00  1.08852  1.08883  1.08852  1.08883         18   
54 2023-12-01 00:54:00  1.08885  1.08905  1.08884  1.08885         37   
55 2023-12-01 00:55:00  1.08887  1.08893  1.08884  1.08887         68   
56 2023-12-01 00:56:00  1.08885  1.08901  1.08882  1.08884         60   
57 2023-12-01 00:57:00  1.08883  1.08887  1.08878  1.08887         24   
58 2023-12-01 00:58:00  1.08888  1.08899  1.08886  1.08896         38   
59 2023-12-01 00:59:00  1.08895  1.08898  1.08893  1.08894         50   

         atr       SMA       EMA  
50  0.000160  1.088622  1.088622  
51  0.000157  1.088622  1.088624  
52  0.000153  1.088624  1.088625  
53  0.000150  1.088646  1.088662  
54  0.000148  1.088668  1.088696  
55  0.000146  1.088693  1.088728  
56  0.000145  1.088715  1.088748  
57  0.000144  1.088741  1.088770  
58  0.000142  1.088775  1.088805  
59  0.000140  1.088806  1.088829

In [39]:
ma = 10
uptrends =   ma*[np.nan] + [ sum([ int(m.chart[i].isBullish()) for i in range(k-m.ma, k) ]) > int(ma*0.75) for k in range(ma, n_candles) ]
downtrends = ma*[np.nan] + [ sum([ int(m.chart[i].isBearish()) for i in range(k-m.ma, k) ]) > int(ma*0.75) for k in range(ma, n_candles) ]

m.data["Uptrend"] = pd.Series(uptrends)
m.data["Downtrend"] = pd.Series(downtrends)

In [40]:
m.data

ts     open     high      low    close  <TICKVOL>  \
0     2023-11-30 17:00:00  1.08845  1.08845  1.08845  1.08845          1   
1     2023-11-30 17:01:00  1.08844  1.08877  1.08768  1.08808         11   
2     2023-11-30 17:02:00  1.08808  1.08821  1.08764  1.08820         24   
3     2023-11-30 17:03:00  1.08830  1.08830  1.08820  1.08820          2   
4     2023-11-30 17:04:00  1.08830  1.08835  1.08830  1.08835          2   
...                   ...      ...      ...      ...      ...        ...   
15517 2023-12-15 11:39:00  1.09091  1.09092  1.09074  1.09090        231   
15518 2023-12-15 11:40:00  1.09090  1.09098  1.09079  1.09084        250   
15519 2023-12-15 11:41:00  1.09084  1.09091  1.09062  1.09072        251   
15520 2023-12-15 11:42:00  1.09073  1.09075  1.09049  1.09068        262   
15521 2023-12-15 11:43:00  1.09070  1.09072  1.09055  1.09056        228   

            SMA       EMA Uptrend Downtrend                 ts2  
0           NaN  1.088450     NaN       NaN 2023-11-30 10:00:00  
1           NaN  1.088508     NaN       NaN 2023-11-30 10:01:00  
2           NaN  1.088454     NaN       NaN 2023-11-30 10:02:00  
3           NaN  1.088426     NaN       NaN 2023-11-30 10:03:00  
4           NaN  1.088412     NaN       NaN 2023-11-30 10:04:00  
...         ...       ...     ...       ...                 ...  
15517  1.091462  1.091359   False     False 2023-12-15 04:39:00  
15518  1.091411  1.091290   False     False 2023-12-15 04:40:00  
15519  1.091344  1.091221   False     False 2023-12-15 04:41:00  
15520  1.091251  1.091136    True     False 2023-12-15 04:42:00  
15521  1.091164  1.091060    True     False 2023-12-15 04:43:00  

[15522 rows x 11 columns]

In [41]:
df["smooth_close"] = savgol_filter(df["close"], 30, 5)

In [42]:
df1 = df[ (df["datetime"] > datetime.datetime(2023, 12, 1, 0, 0, 0)) & (df["datetime"] < datetime.datetime(2023, 12, 1, 4, 0, 0))]

In [43]:
df1

datetime     open     high      low    close  <TICKVOL>  \
1   2023-12-01 00:01:00  1.08844  1.08877  1.08768  1.08808         11   
2   2023-12-01 00:02:00  1.08808  1.08821  1.08764  1.08820         24   
3   2023-12-01 00:03:00  1.08830  1.08830  1.08820  1.08820          2   
4   2023-12-01 00:04:00  1.08830  1.08835  1.08830  1.08835          2   
5   2023-12-01 00:05:00  1.08830  1.08847  1.08820  1.08834         12   
..                  ...      ...      ...      ...      ...        ...   
235 2023-12-01 03:55:00  1.09068  1.09072  1.09064  1.09070         19   
236 2023-12-01 03:56:00  1.09072  1.09073  1.09060  1.09064         21   
237 2023-12-01 03:57:00  1.09065  1.09075  1.09065  1.09067         18   
238 2023-12-01 03:58:00  1.09065  1.09070  1.09062  1.09068         20   
239 2023-12-01 03:59:00  1.09069  1.09079  1.09068  1.09069         25   

          atr       SMA       EMA  smooth_close  
1         NaN       NaN  1.088383      1.088284  
2         NaN       NaN  1.088350      1.088282  
3         NaN       NaN  1.088322      1.088273  
4         NaN       NaN  1.088327      1.088260  
5         NaN       NaN  1.088330      1.088247  
..        ...       ...       ...           ...  
235  0.000099  1.090785  1.090774      1.090684  
236  0.000098  1.090759  1.090750      1.090687  
237  0.000097  1.090742  1.090735      1.090702  
238  0.000096  1.090720  1.090725      1.090721  
239  0.000095  1.090713  1.090719      1.090743  

[239 rows x 10 columns]

In [44]:
df1[df1['close'] > 1.089].shape[0]

174

In [29]:
atr = df1["atr"].iloc[-1]
peaks_idx, _ = find_peaks(df1["smooth_close"], distance = 15, width=3, prominence=atr)
troughs_idx, _ = find_peaks(-1*df1["smooth_close"], distance = 15, width=3, prominence=atr)

In [30]:
fig = ly.Figure(data=ly.Candlestick(x=df1['datetime'],
                            open=df1['open'],
                            high=df1['high'],
                            low=df1['low'],
                            close=df1['close']),
                        )


# for k in peaks_idx:
#     fig.add_scatter(x=[df1["datetime"].iloc[k]], y=[df1["close"].iloc[k]],
#                                     showlegend=False, line=dict(color="blue"))
# for k in troughs_idx:
#     fig.add_scatter(x=[df1["datetime"].iloc[k]], y=[df1["close"].iloc[k]],
#                                     showlegend=False, line=dict(color="black"))

fig.show()

In [ ]:
datetime(2023, 12, 3, 0, 0, 0) += datetime.timedelta(days=10)

NotImplemented

In [ ]:
filter_condition = (df['datetime'] > datetime(2023, 12, 4, 0, 0, 0)) & (df['datetime'] < datetime(2023, 12, 5, 0, 0, 0))

# Apply the filter to the DataFrame
filtered_df = df[filter_condition]

NameError: name 'df' is not defined

In [ ]:
from main import Candle, Market

m = Market(".\\data\\EURUSD_M1_231201_231215.csv")

In [ ]:
m.data["datetime"] > (datetime(2023, 12, 5, 9, 0, 0))

0        False
1        False
2        False
3        False
4        False
         ...  
15517     True
15518     True
15519     True
15520     True
15521     True
Name: <DATETIME>, Length: 15522, dtype: bool

In [ ]:
m.fvg["bull"][:20]

[3,
 53,
 54,
 58,
 63,
 80,
 85,
 86,
 97,
 98,
 112,
 120,
 121,
 122,
 124,
 125,
 129,
 130,
 143,
 144]

In [ ]:
m.fvg_t

{'bull': [Timestamp('2023-12-01 00:03:00'),
  Timestamp('2023-12-01 00:53:00'),
  Timestamp('2023-12-01 00:54:00'),
  Timestamp('2023-12-01 00:58:00'),
  Timestamp('2023-12-01 01:03:00'),
  Timestamp('2023-12-01 01:20:00'),
  Timestamp('2023-12-01 01:25:00'),
  Timestamp('2023-12-01 01:26:00'),
  Timestamp('2023-12-01 01:37:00'),
  Timestamp('2023-12-01 01:38:00'),
  Timestamp('2023-12-01 01:52:00'),
  Timestamp('2023-12-01 02:00:00'),
  Timestamp('2023-12-01 02:01:00'),
  Timestamp('2023-12-01 02:02:00'),
  Timestamp('2023-12-01 02:04:00'),
  Timestamp('2023-12-01 02:05:00'),
  Timestamp('2023-12-01 02:09:00'),
  Timestamp('2023-12-01 02:10:00'),
  Timestamp('2023-12-01 02:23:00'),
  Timestamp('2023-12-01 02:24:00'),
  Timestamp('2023-12-01 02:25:00'),
  Timestamp('2023-12-01 02:32:00'),
  Timestamp('2023-12-01 02:33:00'),
  Timestamp('2023-12-01 02:41:00'),
  Timestamp('2023-12-01 02:44:00'),
  Timestamp('2023-12-01 02:51:00'),
  Timestamp('2023-12-01 02:53:00'),
  Timestamp('2023-12

In [ ]:
df = m.data[ (m.data["datetime"] > datetime(2023, 12, 1, 9, 0, 0)) & (m.data["datetime"] < datetime(2023, 12, 1, 12, 0, 0)) ]

In [ ]:
df

<DATETIME>   <OPEN>   <HIGH>    <LOW>  <CLOSE>  <TICKVOL>  \
541 2023-12-01 09:01:00  1.09090  1.09090  1.09075  1.09079         25   
542 2023-12-01 09:02:00  1.09082  1.09088  1.09069  1.09069         36   
543 2023-12-01 09:03:00  1.09064  1.09084  1.09064  1.09084         42   
544 2023-12-01 09:04:00  1.09079  1.09090  1.09078  1.09080         37   
545 2023-12-01 09:05:00  1.09079  1.09086  1.09075  1.09079         45   
..                  ...      ...      ...      ...      ...        ...   
715 2023-12-01 11:55:00  1.08999  1.09008  1.08990  1.09006         45   
716 2023-12-01 11:56:00  1.09008  1.09009  1.08987  1.08997         33   
717 2023-12-01 11:57:00  1.08996  1.08997  1.08979  1.08981         40   
718 2023-12-01 11:58:00  1.08980  1.08980  1.08966  1.08974         39   
719 2023-12-01 11:59:00  1.08971  1.08980  1.08970  1.08979         50   

          SMA       EMA Uptrend Downtrend  
541  1.091053  1.090999   False     False  
542  1.091026  1.090977   False     False  
543  1.090995  1.090952   False     False  
544  1.090966  1.090943   False     False  
545  1.090935  1.090928   False     False  
..        ...       ...     ...       ...  
715  1.090230  1.090228   False     False  
716  1.090214  1.090203   False     False  
717  1.090206  1.090161   False     False  
718  1.090166  1.090095   False     False  
719  1.090124  1.090041   False     False  

[179 rows x 10 columns]

In [ ]:
for k in m.fvg["bull"]:
    print(k)
    n = df.shape[0]
    if k>=n:
        break
    if k in df.index:
        x=[df['datetime'][k-1], df['datetime'][k-1], df['datetime'][min(k+10, n-1)], df['datetime'][min(k+10, n-1)], df['datetime'][k-1]],
        y=[df['high'][k-1], df['low'][k+1], df['low'][k+1], df['high'][k-1], df['high'][k-1]]

3
53
54
58
63
80
85
86
97
98
112
120
121
122
124
125
129
130
143
144
145
152
153
161
164
171
173
181


In [18]:
m.plot(start=datetime(2023, 12, 1, 9, 0, 0), end=datetime(2023, 12, 1, 12, 0, 0), FVG = True)

TypeError: 'module' object is not callable

In [ ]:
m.getMSSs()

{'bull': [802,
  844,
  845,
  846,
  988,
  1023,
  1097,
  1098,
  1133,
  1134,
  1400,
  1401,
  1670,
  1673,
  1697,
  1698,
  1828,
  1900,
  1901,
  1935,
  2046,
  2151,
  2248,
  2332,
  2632,
  2715,
  2729,
  2826,
  2827,
  2828,
  2968,
  3400,
  3517,
  3533,
  3577,
  3652,
  4027,
  4028,
  4029,
  4255,
  4256,
  5254,
  5329,
  5330,
  5331,
  5689,
  5917,
  5918,
  6266,
  6267,
  6268,
  7246,
  7480,
  8143,
  8237,
  8547,
  8675,
  8704,
  8780,
  9176,
  9177,
  9258,
  9259,
  9366,
  9529,
  9791,
  9792,
  9908,
  9931,
  10122,
  10184,
  10185,
  10391,
  10695,
  10780,
  10793,
  10851,
  11010,
  11054,
  11404,
  11927,
  12006,
  12007,
  12127,
  12170,
  12171,
  12222,
  12223,
  12324,
  12797],
 'bear': [75,
  294,
  13558,
  13641,
  13770,
  13771,
  13863,
  13948,
  14015,
  14169,
  14170,
  14527,
  14633,
  14634,
  14635,
  14858,
  14859,
  15103,
  15104,
  15105,
  15252,
  15253,
  15295,
  15390,
  15490]}